In [1]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from datetime import datetime
import os, sys, pickle, json
import pandas as pd
pd.set_option("display.max_columns", None)

from astropy.coordinates import SkyCoord
from lstchain.reco.utils import  get_effective_time

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# ============================ #
# dirs
objects_dir = "/fefs/aswg/workspace/juan.jimenez/systematics/objects"
# dl2 to dl3 information
dl2dl3_dict = "objects/dict_dl2_dl3_MCs.json"
# output file name
output_dict = "objects/_dict_dl2_cosmics.pkl"
# ============================ #

In [2]:
# reading objects
with open(dl2dl3_dict, "r") as json_file:
    dict_runs = json.load(json_file)

dict_runs[list(dict_runs.keys())[0]]

{'dl2': '/fefs/aswg/workspace/abelardo.moralejo/Crab_performance_paper/data_v0.9.9/DL2/process_with_mc_theta_10.0_az_102.199/dl2_LST-1.Run03319.h5',
 'irf': '/fefs/aswg/workspace/juan.jimenez/data/lst1_systematics/_irf/irf_mc_theta_10.0_az_102.199.fits.gz',
 'dl3': '/fefs/aswg/workspace/juan.jimenez/data/lst1_systematics/_dl3/dl3_LST-1.Run03319.fits.gz'}

In [3]:
%%time
dict_dl2 = {}

for i, run in enumerate(dict_runs.keys()):
    
    print(f"Iterating... {i/len(dict_runs.keys())*100:4.1f}%")
    
    _tab = pd.read_hdf(dict_runs[run]["dl2"], key='dl2/event/telescope/parameters/LST_LSTCam')
    
    # getting only cosmics and applying a small cut in gammaness
#     _tab.query("event_type == 32 and gammaness > 0.05", inplace=True)
    _tab.query("event_type == 32", inplace=True)
    
    eff_t, el_t = get_effective_time(_tab)
    
    tmp_dict = {
        "effective_time" : eff_t.to(u.s).value,
        "elapsed_time" : el_t.to(u.s).value,
        "zd" : np.array(90 - np.rad2deg(_tab["alt_tel"])),
        "az" : np.array(np.rad2deg(_tab["az_tel"])),
        "intensity" : np.array(_tab["intensity"])
    }

    dict_dl2[int(run)] = tmp_dict

Iterating...  0.0%
Iterating...  0.9%
Iterating...  1.7%
Iterating...  2.6%
Iterating...  3.4%
Iterating...  4.3%
Iterating...  5.1%
Iterating...  6.0%
Iterating...  6.8%
Iterating...  7.7%
Iterating...  8.5%
Iterating...  9.4%
Iterating... 10.3%
Iterating... 11.1%
Iterating... 12.0%
Iterating... 12.8%
Iterating... 13.7%
Iterating... 14.5%
Iterating... 15.4%
Iterating... 16.2%
Iterating... 17.1%
Iterating... 17.9%
Iterating... 18.8%
Iterating... 19.7%
Iterating... 20.5%
Iterating... 21.4%
Iterating... 22.2%
Iterating... 23.1%
Iterating... 23.9%
Iterating... 24.8%
Iterating... 25.6%
Iterating... 26.5%
Iterating... 27.4%
Iterating... 28.2%
Iterating... 29.1%
Iterating... 29.9%
Iterating... 30.8%
Iterating... 31.6%
Iterating... 32.5%
Iterating... 33.3%
Iterating... 34.2%
Iterating... 35.0%
Iterating... 35.9%
Iterating... 36.8%
Iterating... 37.6%
Iterating... 38.5%
Iterating... 39.3%
Iterating... 40.2%
Iterating... 41.0%
Iterating... 41.9%
Iterating... 42.7%
Iterating... 43.6%
Iterating...

In [4]:
pd.read_hdf(dict_runs[list(dict_runs.keys())[0]]["dl2"], key='dl2/event/telescope/parameters/LST_LSTCam')

,obs_id,event_id,intensity,log_intensity,x,y,r,phi,length,length_uncertainty,width,width_uncertainty,psi,skewness,kurtosis,time_gradient,intercept,leakage_intensity_width_1,leakage_intensity_width_2,leakage_pixels_width_1,leakage_pixels_width_2,n_pixels,concentration_cog,concentration_core,concentration_pixel,n_islands,alt_tel,az_tel,calibration_id,dragon_time,ucts_time,tib_time,ucts_jump,mc_type,wl,tel_id,tel_pos_x,tel_pos_y,tel_pos_z,trigger_type,ucts_trigger_type,trigger_time,event_type,delta_t,log_reco_energy,reco_energy,reco_disp_norm,reco_disp_sign,reco_disp_dx,reco_disp_dy,reco_src_x,reco_src_y,signed_time_gradient,signed_skewness,reco_alt,reco_az,reco_type,gammaness
0,3319,1,5175.563053,3.713958,0.028399,0.040546,0.049502,0.959787,1.122596,0.007710,1.112845,0.008035,-0.518826,-0.136247,1.976222,6.447608,14.478188,0.048741,0.127929,0.015094,0.043127,632,0.003409,0.240564,0.007029,42,1.354214,2.121335,0,1.608250e+09,0,0,False,-9999,0.991315,1,50.0,50.0,16.0,1,1,1.608250e+09,32,0.000000,0.092887,1.238475,0.352497,-1.0,-0.306109,0.174790,-0.277710,0.215336,6.447608,-0.136247,1.344168,2.155566,101,0.014099
1,3319,2,296.229436,2.471628,0.044439,0.286822,0.290245,1.417083,0.126657,0.003997,0.092432,0.003311,-0.420219,-0.147140,2.180271,-1.217166,16.767604,0.000000,0.000000,0.000000,0.000000,22,0.264913,0.351448,0.123187,1,1.354214,2.121335,0,1.608250e+09,0,0,False,-9999,0.729786,1,50.0,50.0,16.0,1,42,1.608250e+09,32,0.000210,-1.135448,0.073207,0.247030,1.0,0.225538,-0.100778,0.269977,0.186044,1.217166,0.147140,1.363751,2.153661,101,0.150257
2,3319,3,132.526345,2.122302,-0.591048,0.455720,0.746337,2.484763,0.102198,0.004269,0.041113,0.001454,-1.223130,-0.039517,1.924927,2.300622,16.273114,0.000000,0.000000,0.000000,0.000000,8,0.680619,0.265106,0.265106,1,1.354214,2.121335,0,1.608250e+09,0,0,False,-9999,0.402288,1,50.0,50.0,16.0,1,42,1.608250e+09,32,0.000844,-1.441073,0.036218,0.307891,-1.0,-0.104900,0.289470,-0.695948,0.745191,2.300622,-0.039517,1.327917,2.232176,101,0.319120
3,3319,4,135.749321,2.132738,-0.487272,0.250867,0.548059,2.666144,0.135529,0.006341,0.040870,0.003234,-0.103052,-0.283872,2.188727,-1.220837,17.921368,0.000000,0.000000,0.000000,0.000000,9,0.534413,0.460000,0.234572,1,1.354214,2.121335,0,1.608250e+09,0,0,False,-9999,0.301557,1,50.0,50.0,16.0,1,1,1.608250e+09,32,0.000035,-1.414833,0.038474,0.324380,1.0,0.322659,-0.033369,-0.164613,0.217498,1.220837,0.283872,1.348202,2.156528,101,0.495424
4,3319,6,121.752949,2.085479,0.652095,-0.937659,1.142117,-0.963130,0.059958,0.002906,0.042070,0.001641,-0.657356,-0.272272,2.144317,4.156472,16.711540,0.589826,1.000000,0.001617,0.002695,5,0.805784,0.000000,0.306455,1,1.354214,2.121335,0,1.608250e+09,0,0,False,-9999,0.701654,1,50.0,50.0,16.0,1,42,1.608250e+09,32,0.000101,-0.956831,0.110451,0.233033,-1.0,-0.184471,0.142389,0.467625,-0.795270,4.156472,-0.272272,1.368941,1.979228,101,0.147729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4744234,3319,5174215,200.950094,2.303088,0.027961,0.117017,0.120311,1.336247,0.086402,0.003100,0.061211,0.002357,-0.782835,0.111852,2.034677,-2.295654,17.439926,0.000000,0.000000,0.000000,0.000000,10,0.467589,0.173761,0.173761,1,1.412812,2.425186,0,1.608252e+09,0,0,False,-9999,0.708446,1,50.0,50.0,16.0,1,0,1.608252e+09,32,0.000143,-1.293552,0.050868,0.249383,-1.0,-0.176792,0.175888,-0.148831,0.292905,-2.295654,0.111852,1.407165,2.489444,101,0.096032
4744235,3319,5174216,119.938271,2.078958,-0.920364,-0.524740,1.059444,-2.623416,0.108104,0.005215,0.034291,0.002467,-1.175518,0.115901,2.116413,-0.970704,18.186383,0.879347,1.000000,0.002695,0.003774,7,0.504224,0.504224,0.304444,1,1.412812,2.425186,0,1.608252e+09,0,0,False,-9999,0.317200,1,50.0,50.0,16.0,1,0,1.608252e+09,32,0.000431,-1.000160,0.099963,0.286166,-1.0,-0.110192,0.264100,-1.030557,-0.260640,-0.9707

In [4]:
%%time
# Saving the objects
with open(output_dict, 'wb') as f:
    pickle.dump(dict_dl2, f, pickle.HIGHEST_PROTOCOL)
    
# # this to read the objects
# with open(output_dict, 'rb') as f:
#     dict_dl2 = pickle.load(f)

CPU times: user 270 ms, sys: 19.9 s, total: 20.2 s
Wall time: 1min 51s
